<a href="https://colab.research.google.com/github/AswathiViswam/music_recommender/blob/main/music_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🎵 Music Mood-Based YouTube Recommendation API using FastAPI

## Step 1: Install Required Libraries
!pip install fastapi[all] pyngrok pydub ffmpeg-python google-api-python-client librosa soundfile

## Step 2: Set Your ngrok Auth Token
NGROK_AUTH_TOKEN = ""
!ngrok config add-authtoken $NGROK_AUTH_TOKEN

## Step 3: Define FastAPI App with Audio Upload
from fastapi import FastAPI, UploadFile, File
from googleapiclient.discovery import build
from pydub import AudioSegment
import shutil, librosa, numpy as np, os

app = FastAPI()

def convert_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_file(mp3_path)
    audio.export(wav_path, format="wav")

def extract_audio_features(path):
    y, sr = librosa.load(path, duration=30)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    avg_chroma = np.mean(chroma)
    if tempo > 130 and avg_chroma > 0.6:
        return "energetic dance music"
    elif tempo < 80:
        return "slow emotional songs"
    elif avg_chroma < 0.4:
        return "lofi chill beats"
    else:
        return "pop music"

def youtube_search(query):
    YOUTUBE_API_KEY = ""  # ✅ Moved inside the function
    youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)
    req = youtube.search().list(q=query, part="snippet", maxResults=5, type="video")
    res = req.execute()
    return [
        {
            "title": item['snippet']['title'],
            "url": f"https://www.youtube.com/watch?v={item['id']['videoId']}"
        } for item in res['items']
    ]

@app.post("/recommend")
async def recommend_music(file: UploadFile = File(...)):
    try:
        filename = file.filename
        with open(filename, "wb") as f:
            shutil.copyfileobj(file.file, f)
        if filename.endswith(".mp3"):
            wav_path = filename.replace(".mp3", ".wav")
            convert_to_wav(filename, wav_path)
            os.remove(filename)
            filename = wav_path
        mood = extract_audio_features(filename)
        videos = youtube_search(mood)
        return {"detected_mood": mood, "recommended_videos": videos}
    except Exception as e:
        return {"error": str(e)}

## Step 4: Run FastAPI with ngrok
from pyngrok import ngrok
import uvicorn, threading

# Kill any existing tunnels and avoid port conflict
ngrok.kill()
!fuser -k 8000/tcp || true

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()
public_url = ngrok.connect(8000)
print("Your API is live at:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


INFO:     Started server process [8296]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Your API is live at: NgrokTunnel: "https://03ab-34-150-149-110.ngrok-free.app" -> "http://localhost:8000"
